In [1]:
import json
import os
from tqdm import tqdm
from multiprocessing import Pool
import sys
import time
sys.path.append('/home/rahul/ripper/GitRipper')

import pandas as pd
from GitRipper.Ripper import collect

In [2]:
cpu_count = os.cpu_count()
print(cpu_count)

8


In [3]:
#Load the json file
with open('/home/rahul/ripper/GitRipper/API_keys.json') as f:
    keys = json.load(f)['keys']

In [4]:
columns = ['repo_owner', 'repo_name', 'oid', 'messageHeadline', 'author_name',
       'author_email', 'author_user_login', 'author_user_location',
       'author_user_company', 'author_user_pronouns', 'author_user_bio',
       'author_user_websiteUrl', 'author_user_twitterUsername', 'author_date',
       'additions', 'deletions']

In [5]:
#create an empty dataframe and save it to a csv
if os.path.exists('/home/rahul/oss/code/Codegov_Commits.csv'):
    pass
else:
    df = pd.DataFrame(columns=columns)
    df.to_csv('/home/rahul/oss/code/Codegov_Commits.csv', index=False)

In [6]:
#get all the packages alrady collected
slugs_collected = []
for df in pd.read_csv('/home/rahul/oss/code/Codegov_Commits.csv', chunksize=1000):
    repo_names = df['repo_name'].tolist()
    repo_owners = df['repo_owner'].tolist()
    for i in range(len(repo_names)):
        slugs_collected.append(repo_owners[i] + '/' + repo_names[i])
    slugs_collected = list(set(slugs_collected))

In [7]:
keys_list = [(i.split(":")[0], i.split(":")[1]) for i in keys]
collector = collect(keys_list)

In [8]:
parallelcount = cpu_count

--------------------------------------------
### Getting the input data ready

In [26]:
import json
import requests
from pandas import json_normalize
from sqlalchemy import create_engine, engine, text, types, MetaData, Table, String
import pandas as pd
import numpy as np
from datetime import datetime

In [27]:
# Create SQLAlchemy engine to connect to MySQL Database

from sqlalchemy import create_engine

engine = create_engine(config.SQL_ALCHEMY_STRING)

In [29]:
# Query to fetch all records

import pandas as pd

from sqlalchemy.sql import text
sql = '''
    SELECT * FROM 08_code_github_repos;
'''
with engine.connect().execution_options(autocommit=True) as conn:
    query = conn.execute(text(sql))         
code_df = pd.DataFrame(query.fetchall())

code_df.to_csv('/home/rahul/oss/code/code_repos.csv')

---------------------------------

In [30]:
def get_owner_and_repo(url):
    ### if it has .git, remove it
    if url.endswith(".git"):
        url = url[:-4]
    ### get owner and repo
    owner = url.split("/")[-2]
    repo = url.split("/")[-1]
    return owner, repo

In [31]:
def get_commits(owner, repo, key, since):
    try:
        if owner + '/' + repo in slugs_collected:
            return None
        commits = collector.collectCommits(owner, repo, key, since)
        return commits
    except Exception as e:
        print("Error: ", e)
        return None

In [32]:
overall_it = int(pd.read_csv("/home/rahul/oss/code/code_repos.csv").shape[0]/parallelcount)

In [34]:
for df in tqdm(pd.read_csv("/home/rahul/oss/code/code_repos.csv", chunksize=parallelcount), total=overall_it):
    slugs = [get_owner_and_repo(i) for i in df.RepoURL]
    owners = [i[0] for i in slugs]
    repos = [i[1] for i in slugs]
    best_keys = collector.getBestKeys(n=parallelcount)
    sinces = ["2007-01-01T00:00:00Z" for i in range(parallelcount)]
    args = list(zip(owners, repos, best_keys, sinces))
    with Pool(parallelcount) as p:
        result = p.starmap(get_commits, args)
    result = [i for i in result if i is not None]
    if len(result) == 0:
        continue
    df_result = pd.concat(result)[columns]
    df_result.to_csv('/home/rahul/oss/code/Codegov_Commits.csv', mode='a', header=False, index=False)

  1%|          | 7/759 [03:23<2:44:19, 13.11s/it]  

Error:  'NoneType' object is not subscriptable


  1%|          | 8/759 [03:31<2:26:46, 11.73s/it]

Error:  'NoneType' object is not subscriptable


  2%|▏         | 14/759 [04:20<2:20:43, 11.33s/it]

...

100%|█████████▉| 758/759 [12:44:22<01:57, 117.70s/it]

Error:  'data'
Error:  'data'


100%|██████████| 759/759 [12:49:10<00:00, 168.98s/it]

Error:  'NoneType' object is not subscriptable


760it [12:49:12, 60.73s/it]                          
